In [ ]:
using SQLite
using StatsPlots; gr() # choose your favourite plotting backend
using PlotMGA
using DataFrames

In [ ]:
path = "example.sqlite"

db = SQLite.DB(path)

total_costs = get_param(db, :total_costs, 
    [:mga_iteration, :alternative, :total_value])
storages_invested = get_param(db, :storages_invested, 
    [:mga_iteration, :alternative, :technology, :installation, :total_value])
units_invested = get_param(db, :units_invested, 
    [:mga_iteration, :alternative, :technology, :installation, :total_value])
unit_flows = get_param(db, :unit_flow, 
    [:mga_iteration, :alternative, :technology, :installation, :destination, :total_value])

SQLite.close(db)

In [ ]:
picture_folder = "pics/"

In [ ]:
commodity_types = [
    ProfileType("Electricity", :destination=>ByRow(contains("Electricity"))),
    ProfileType("Heat", :destination=>ByRow(contains("Space_Heating")))
]

storages = unique(storages_invested.technology)
production_types = [
    ProfileType("Production", :technology=>ByRow(!in(storages))),
    ProfileType("Storage", :technology=>ByRow(in(storages))),
]

plot_all_mga_profiles(
    unit_flows, PlotMGA.GWh, commodity_types, production_types,
    fig_prefix="$(picture_folder)mga_profile_for_"
)

In [ ]:
markers = Dict(
    "fuzzy_1h.json" => :x,
    "hsj_05_1h.json" => :circle,
    "hsj_1_1h.json" => :rect,
    "Optimal" => :diamond
)

plot_technology_interactions(
    units_invested, total_costs, markers, 
    fig_prefix="$(picture_folder)technology_interactions_", display_plot=false
)

In [ ]:
plot_technology_effects(
    units_invested, total_costs, 
    fig_prefix="$(picture_folder)technology_effects_on_cost_"
)